In [ ]:
import os
import sys
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib as mpl
from scipy.stats import pearsonr

In [ ]:
company = ''

# fill in the path to tasks performance outputs from WP3 performance_evaluation.py code
# or task perf (with significance results) from performance evaluations CLS_vs_CLSAUX
config = {
    "SPCLS_SPCLSAUX":"< path to task performance with significance results from cls_vs_clsaux : spcls_vs_spclsaux >",
    "MPCLS_MPCLSAUX":"< path to task performance with significance results from cls_vs_clsaux : mpcls_vs_mpclsaux >",
    "MPCLS_SPCLSAUX":"< path to task performance with significance results from cls_vs_clsaux : mpcls_vs_spclsaux >" ,
    "SPCLS_MPCLSAUX":"< path to task performance with significance results from cls_vs_clsaux : spcls_vs_mpclsaux >",
    "SPCLS_MPCLS":"< path to performance_evluation.py task perf deltas of comparison SPCLS-MPCLS cls/deltas/deltas_per-task_performances_NOUPLOAD.csv > ",
    "SPCLSAUX_MPCLSAUX":"< path to performance_evluation.py task perf deltas of comparison SPCLSAUX-MPCLSAUX clsaux/deltas/deltas_per-task_performances_NOUPLOAD.csv >"
}

In [ ]:
def tversky(a, b, alpha=0.95, beta=0.05):
    
    sum_a = a.sum()
    sum_b = b.sum()

    num_common = np.logical_and(a, b).sum()
    tversky_ab = num_common / (alpha * sum_a + beta * sum_b )
    tversky_ba = num_common / (beta * sum_a + alpha * sum_b )

    return tversky_ab, tversky_ba

In [ ]:
# load in all the required files + merge significance columns

t8c_clsaux = pd.read_csv("/db/melloddy/DATA/Internal_Y3/Melloddy_tuner_io/outputs_MT-3.0.2_w_aux_hts_PLsppl2a/results_tmp/classification/T8c.csv", usecols=['cont_classification_task_id', 'classification_task_id', 'input_assay_id', 'threshold']).dropna(subset=['cont_classification_task_id'])
t8c = pd.read_csv("/db/melloddy/DATA/Internal_Y3/Melloddy_tuner_io/outputs_MT-3.0.2/results_tmp/classification/T8c.csv", usecols=['cont_classification_task_id', 'classification_task_id', 'input_assay_id', 'threshold']).dropna(subset=['cont_classification_task_id'])
t8c['cont_classification_task_id'] = t8c['cont_classification_task_id'].astype(int)
t8c_clsaux['cont_classification_task_id'] = t8c_clsaux['cont_classification_task_id'].astype(int)

# map clsuax to cls task ids
cols=['cont_classification_task_id', 'cont_classification_task_id_clsaux']
clsaux2cls = t8c.merge(t8c_clsaux, on=['input_assay_id', 'threshold'], suffixes=('', '_clsaux'))[cols]
clsaux2cls = clsaux2cls.set_index('cont_classification_task_id_clsaux')['cont_classification_task_id'].to_dict()


cols = ['cont_classification_task_id','roc_auc_score', 'SP_significant', 'SP_pvalue','MP_significant', 'MP_pvalue']
spcls_mpcls = t8c.merge(pd.read_csv(config["SPCLS_MPCLS"]), on='classification_task_id')[cols].rename(columns={'MP_significant':'sign SPCLS->MPCLS', 'cont_classification_task_id':'task_id'})
spclsaux_mpclsaux = t8c_clsaux.merge(pd.read_csv(config["SPCLSAUX_MPCLSAUX"]), on='classification_task_id')[cols].rename(columns={'MP_significant':'sign SPCLSAUX->MPCLSAUX', 'cont_classification_task_id':'task_id'})
spclsaux_mpclsaux['task_id'] = spclsaux_mpclsaux['task_id'].map(clsaux2cls)

spcls_spclsaux = pd.read_csv(config["SPCLS_SPCLSAUX"]).rename(columns={'significant clsaux > cls':'sign SPCLS->SPCLSAUX'})
mpcls_mpclsaux = pd.read_csv(config["MPCLS_MPCLSAUX"]).rename(columns={'significant clsaux > cls':'sign MPCLS->MPCLSAUX'})
spcls_mpclsaux = pd.read_csv(config['SPCLS_MPCLSAUX']).rename(columns={'significant clsaux > cls':'sign SPCLS->MPCLSAUX'})

cols=['task_id', 'sign SPCLS->MPCLS', 'sign SPCLS->SPCLSAUX', 'sign MPCLS->MPCLSAUX','sign SPCLS->MPCLSAUX', 'sign SPCLSAUX->MPCLSAUX']
merge = spcls_mpcls.merge(spcls_spclsaux, on='task_id').merge(mpcls_mpclsaux, on='task_id').merge(spcls_mpclsaux, on='task_id').merge(spclsaux_mpclsaux, on='task_id')[cols]

# diamond comparisons nomenclature

1. sign boost in SPCLS -> SPCLSAUX
2. sign boost in SPCLS -> MPCLS
3. sign boost in SPCLS -> MPCLSAUX
4. sign boost in SPCLS -> SPCLSAUX and/or SPCLS -> MPCLS

In [ ]:
frac_sign_1 = merge['sign SPCLS->SPCLSAUX'].sum()/merge.shape[0]
frac_sign_2 = merge['sign SPCLS->MPCLS'].sum()/merge.shape[0]
frac_sign_3 = merge['sign SPCLS->MPCLSAUX'].sum()/merge.shape[0]
frac_sign_4 = np.logical_or(merge['sign SPCLS->SPCLSAUX'], merge['sign SPCLS->MPCLS']).sum()/merge.shape[0]

frac_sign_1, frac_sign_2, frac_sign_3, frac_sign_4

# Is the effect additive between SPCLS->MPCLS and SPCLS->SPCLSAUX?
how many of the tasks significantly boosted by aux data in SP setup are still sign. boosted when going to MP? 

In [ ]:
frac_1_in_3, frac_3_in_1 =  tversky(merge['sign SPCLS->SPCLSAUX'], merge['sign SPCLS->MPCLSAUX'], alpha=1, beta=0)
frac_1_in_3, frac_3_in_1

how many of the tasks sign. boosted in SPCLS->MPCLS setup are still sign. boosted when going to MPCLSAUX? 

In [ ]:
frac_2_in_3, frac_3_in_2 = tversky(merge['sign SPCLS->MPCLS'],merge['sign SPCLS->MPCLSAUX'], alpha=1, beta=0)
frac_2_in_3, frac_3_in_2

how many tasks get boosted in SPCLS->MPCLSAUX but are not boosted by SPCLS-SPCLSAUX and not by SPCLS->MPCLS?

In [ ]:
scls_spclsaux_OR_spcls_mpcls = np.logical_or(merge['sign SPCLS->SPCLSAUX'], merge['sign SPCLS->MPCLS'])
frac_4_in_3, frac_3_in_4 = tversky(scls_spclsaux_OR_spcls_mpcls, merge['sign SPCLS->MPCLSAUX'], alpha=1, beta=0)
frac_4_in_3, frac_3_in_4

# -> not additive effect since only 100-X% of tasks boosted either by SPCLS->SPCLSAUX or in SPCLS->MPCLS are also boosted in SPCLS->MPCLSAUX
# this would suggest some loss in significance boost wrt to 4

# -> traction gained from MP synergies, Y% tasks boosted by SPCLS-MPCLSAUX are already boosted in either of the two former setups, W% additional tasks boosted in SPCLS->MPCLSAUX wrt to 4!

In [ ]:
results = pd.DataFrame({'comparison':['SPCLS -> SPCLSAUX', 'SPCLS -> MPCLS', 'SPCLS -> MPCLSAUX', 'SPCLSAUX and/or SPCLS -> MPCLS'], 
                        'total fraction sign boost':[frac_sign_1, frac_sign_2, frac_sign_3, frac_sign_4], 
                        'fraction of comparison in SPCLS -> MPCLSAUX': [1, frac_1_in_3, frac_2_in_3, frac_4_in_3],
                        'fraction of SPCLS -> MPCLSAUX in comparison': [1, frac_3_in_1, frac_3_in_2, frac_3_in_1]
                       }).set_index('comparison')
results

In [ ]:
# share this 
resutls.to_csv(f"{company}_auxdata_additivity.csv", index=None)

What happens to tasks sign boosted in SPCLS-SPCLSAUX that do not get sign boost in SPCLS->MPCLSAUX? <br>
Are their perf significantly hurt or only not sign anymore? <br>


In [ ]:
task_sel = merge.loc[(merge['sign SPCLS->SPCLSAUX']==1)&(merge['sign SPCLS->MPCLSAUX']==0)]['task_id']

metric = 'auroc'
x = f'{metric}_cls'
y = f'{metric}_clsaux'

perf_sel = spcls_mpclsaux.loc[spcls_mpclsaux['task_id'].isin(task_sel)]
plt.scatter(perf_sel[x], perf_sel[y])
plt.plot([0.5, 1], [0.5, 1], ls='--', c='k')
#plt.boxplot((perf_sel[y] - perf_sel[x]).values)
plt.xlabel(x)
plt.ylabel(y)